<a href="https://colab.research.google.com/github/arzuunr/ynbs_is_bankasi/blob/main/ynbs_pazarlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook


In [5]:
data_set=pd.read_excel("/content/gorev2-musteri-listesi-1724442948 (3).xlsx")

In [6]:
data_set.head()

,Seçim Kriterleri
0,Bu kısımda hangi kriterlere göre seçim yaptığı...
1,NaN
2,Toplam Müşteri Sayısı
3,0
4,NaN


## **Seçim Kriterleri**

#### Kampanya, market ve akaryakıt harcaması düşük (0’a yakın, toplam 600 TL’den az), diğer harcamaları aktif (toplam 0 TL’den fazla) ve SMS izni olan müşterilere yöneliktir. Bu, kampanyanın hedef kitlesi olan Bankamız kartları ile yeterli market ve akaryakıt harcaması yapmayan ancak diğer harcama türlerinde aktif müşterileri çekmeyi amaçlar.

#### Yani filtrelemede sms izni olan (1) ve market + akaryakıt harcaması (600tl'den az) olanlar alınmıştır.

In [7]:
 sheet_hepsi = load_workbook(filename = 'gorev2-musteri-listesi-1724442948 (3).xlsx')


In [8]:
print (sheet_hepsi.sheetnames)

['Görev2-Müşteri Listesi', 'Müşteri Bilgileri', 'Harcama Bilgileri']


In [9]:
musteri_bilgileri = pd.read_excel("gorev2-musteri-listesi-1724442948 (3).xlsx", sheet_name="Müşteri Bilgileri")
harcama_bilgileri = pd.read_excel("gorev2-musteri-listesi-1724442948 (3).xlsx", sheet_name="Harcama Bilgileri")

In [12]:
print(harcama_bilgileri.loc[harcama_bilgileri['Harcama Türü'] == "akaryakıt"][['IAB (MusteriNo)','Harcama Türü','Harcama Tutarı']])


        IAB (MusteriNo) Harcama Türü  Harcama Tutarı
12               100000    akaryakıt            4132
14               100000    akaryakıt            2669
28               100000    akaryakıt            5530
48               100001    akaryakıt            6559
64               100001    akaryakıt            3484
...                 ...          ...             ...
125934           102998    akaryakıt               0
125950           102998    akaryakıt               0
125968           102999    akaryakıt               0
125985           102999    akaryakıt               0
125994           102999    akaryakıt               0

[9000 rows x 3 columns]


In [14]:
# Market ve akaryakıt harcamalarını filtrele
market_harcamalar = harcama_bilgileri[harcama_bilgileri['Harcama Türü'] == "market"]
akaryakit_harcamalar = harcama_bilgileri[harcama_bilgileri['Harcama Türü'] == "akaryakıt"]

In [15]:
market_toplam = market_harcamalar.groupby('IAB (MusteriNo)')['Harcama Tutarı'].sum().reset_index()
market_toplam.columns = ["Musteri_ID", "Toplam_Market"]

In [16]:
akaryakit_toplam = akaryakit_harcamalar.groupby('IAB (MusteriNo)')['Harcama Tutarı'].sum().reset_index()
akaryakit_toplam.columns = ["Musteri_ID", "Toplam_Akaryakit"]

In [17]:
market_akaryakit = pd.merge(market_toplam, akaryakit_toplam, on="Musteri_ID", how="outer").fillna(0)

In [18]:
market_akaryakit["Toplam_Market_Akaryakit"] = market_akaryakit["Toplam_Market"] + market_akaryakit["Toplam_Akaryakit"]

In [19]:
dusuk_harcama = market_akaryakit[market_akaryakit["Toplam_Market_Akaryakit"] < 600]
print("Market ve Akaryakıt Harcaması Düşük Olanlar:")
print(dusuk_harcama)

Market ve Akaryakıt Harcaması Düşük Olanlar:
      Musteri_ID  Toplam_Market  Toplam_Akaryakit  Toplam_Market_Akaryakit
7         100007              0                 0                        0
9         100009              0                 0                        0
12        100012              0                 0                        0
15        100015              0                 0                        0
17        100017              0                 0                        0
...          ...            ...               ...                      ...
2991      102991              0                 0                        0
2992      102992              0                 0                        0
2994      102994              0                 0                        0
2995      102995              0                 0                        0
2999      102999              0                 0                        0

[738 rows x 4 columns]


In [20]:
dusuk_harcama_ids = dusuk_harcama["Musteri_ID"].tolist()

df_dh = pd.DataFrame(dusuk_harcama_ids, columns=['IAB (MusteriNo)'])

# Excel dosyasına kaydet
df_dh.to_csv('dusuk_harcama.csv', index=False)

print("dusuk_harcama.csv' dosyasına kaydedildi.")

dusuk_harcama.csv' dosyasına kaydedildi.


In [21]:
import pandas as pd
a=pd.read_csv('dusuk_harcama.csv')
a.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   IAB (MusteriNo)  738 non-null    int64
dtypes: int64(1)
memory usage: 5.9 KB


In [22]:
# Market ve akaryakıt dışındaki harcamaları filtrele
diger_harcamalar = harcama_bilgileri[~harcama_bilgileri['Harcama Türü'].isin(["market", "akaryakıt"])]

# Her müşteri için toplam diğer harcamaları hesapla
diger_toplam = diger_harcamalar.groupby('IAB (MusteriNo)')['Harcama Tutarı'].sum().reset_index()
diger_toplam.columns = ["Musteri_ID", "Toplam_Diger"]

# Diğer harcamaları aktif olanları bul (örneğin toplam 1000 TL'den fazla)
aktif_diger = diger_toplam[diger_toplam["Toplam_Diger"] > 1000]
print("Diğer Harcamaları Aktif Olanlar:")
print(aktif_diger)

Diğer Harcamaları Aktif Olanlar:
      Musteri_ID  Toplam_Diger
0         100000        118016
1         100001        102783
2         100002        153265
3         100003         32609
4         100004        102147
...          ...           ...
2995      102995        108100
2996      102996        121843
2997      102997         83718
2998      102998         43007
2999      102999        123145

[2999 rows x 2 columns]


In [23]:
# SMS izni olan müşterileri filtrele
sms_izinli_musteriler = musteri_bilgileri[musteri_bilgileri["SMS iletişim izni var mı?"] == 1]
#print("SMS İzni Olan Müşteriler:")
#print(sms_izinli_musteriler)
sms_izinli_musteriler.head()

,IAB (MusteriNo),Doğum Yılı,Aylık Gelir,Eğitim Durumu,Evli/Bekar,Cinsiyet,İş durumu,Evi var mı?,Arabası var mı?,İşcep yüklü mü?,SMS iletişim izni var mı?,E-posta iletişim izni var mı?
1,100001,1953,29500,Lisans,B,K,Memur,0,0,1,1,0
2,100002,1999,31000,Lise,B,K,İşçi,1,0,1,1,1
6,100006,1960,215000,Lisans,B,K,İşçi,0,0,0,1,0
9,100009,1968,55000,Lise,B,K,İşçi,0,1,0,1,1
10,100010,1976,25000,Lise,E,E,Memur,1,0,0,1,0


In [67]:
sms_file=sms_izinli_musteriler['IAB (MusteriNo)'].tolist()

df_sms = pd.DataFrame(sms_file, columns=['IAB (MusteriNo)'])

# CSV dosyasına kaydet
df_sms.to_csv('sms_izinli_musteriler.csv', index=False)

print("SMS izni olan müşteriler 'sms_izinli_musteriler.csv' dosyasına kaydedildi.")


SMS izni olan müşteriler 'sms_izinli_musteriler.csv' dosyasına kaydedildi.


In [24]:
musteri_bilgileri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   IAB (MusteriNo)                3000 non-null   int64 
 1   Doğum Yılı                     3000 non-null   int64 
 2   Aylık Gelir                    3000 non-null   int64 
 3   Eğitim Durumu                  3000 non-null   object
 4   Evli/Bekar                     3000 non-null   object
 5   Cinsiyet                       3000 non-null   object
 6   İş durumu                      3000 non-null   object
 7   Evi var mı?                    3000 non-null   int64 
 8   Arabası var mı?                3000 non-null   int64 
 9   İşcep yüklü mü?                3000 non-null   int64 
 10  SMS iletişim izni var mı?      3000 non-null   int64 
 11  E-posta iletişim izni var mı?  3000 non-null   int64 
dtypes: int64(8), object(4)
memory usage: 281.4+ KB


In [25]:
sms_izinli_df = pd.read_csv('sms_izinli_musteriler.csv')
dusuk_harcama_df = pd.read_csv('dusuk_harcama.csv')

# ID sütunlarını al
sms_izinli_ids = sms_izinli_df['IAB (MusteriNo)'].tolist()
dusuk_harcama_ids = dusuk_harcama_df['IAB (MusteriNo)'].tolist()

# Ortak ID'leri bul
ortak_musteriler = list(set(sms_izinli_ids) & set(dusuk_harcama_ids))

# Sonucu yazdır
print(f"Ortak müşteri sayısı: {len(ortak_musteriler)}")
print("Ortak Müşteri ID'leri:")
for musteri_id in ortak_musteriler:
    print(musteri_id)

Ortak müşteri sayısı: 359
Ortak Müşteri ID'leri:
102405
102409
100363
102418
100375
102424
102428
100387
102437
100390
102439
100397
100398
102450
102453
102456
100413
102468
100421
100423
100424
100426
102476
102485
102488
102494
100447
100450
100451
102499
100455
102503
102507
102513
100467
102516
100471
102523
100478
100481
100485
100489
100493
100496
100498
102546
102560
100515
102567
102572
100525
100528
102584
100537
102587
100541
100546
102595
100549
100554
100555
100557
100565
102618
102627
100584
100585
100586
100587
102632
100590
102648
100606
100607
100616
102664
100619
100621
102688
102690
100645
100651
102706
100659
102710
100674
102725
102726
102734
102737
102739
100709
102761
100715
100726
100727
102781
102782
100749
102808
100764
102826
102828
102830
102841
102847
100804
102852
100809
102864
100817
102865
102866
100825
102885
100843
100846
100850
100870
102920
102926
102928
100886
102939
100893
100898
102949
102957
102961
102965
100922
100923
100927
100929
102979
100940

In [26]:
ortak_df = pd.DataFrame(ortak_musteriler, columns=['IAB (MusteriNo)'])
ortak_df.to_csv('ortak_musteriler.csv', index=False)
print("Ortak müşteriler 'ortak_musteriler.csv' dosyasına kaydedildi.")

Ortak müşteriler 'ortak_musteriler.csv' dosyasına kaydedildi.
